### Create a table 1

In [ ]:
import pandas as pd
import numpy as np
import joblib
import glob
import duckdb
import os
import seaborn as sns
import matplotlib.pyplot as plt
from tableone import TableOne


from hemat.config import Config

config = Config(root="C:\\motten\\projects\\hemat", experiment_name="survival")

### Load data

In [ ]:
AUMC_table_1_df=pd.read_csv(os.path.join(config.directory("processed"), "AUMC_table_1_df.csv"))
UMCU_table_1_df=pd.read_csv(os.path.join(config.directory("processed"), "UMCU_table_1_df.csv"))

### Concat dataframes

In [ ]:
AUMC_table_1_df = AUMC_table_1_df[['malignant_hemat','Geslacht','neutrophiles_smaller_than_0.5','autoloog','allogeen','chr_renal_insuf','chron_dialysis','resp_insuf','cardio_vasc_insuf',
                    'cirrhosis','neoplasm','hem_malign','aids','imm_insuf',
                    'DiagnoseCode_last','age_at_admission', 'neutrophiles_count', 'nice_ap4_score','Hoofdgroep','Survival_in_days', 'day_of_admission_max_count', 'mech_vent_max_count', 'rrt_max_count',
       'vasopr_inotr_max_count', 'transfusion_max_count', 'mort_90d', 'mort_1y']]

In [ ]:
AUMC_table_1_df.rename(columns={'Hoofdgroep': 'APACHE_IV_adm_diag_extracted'}, inplace=True)

In [ ]:
UMCU_table_1_df = UMCU_table_1_df[['malignant_hemat','Geslacht','neutrophiles_smaller_than_0.5','autoloog','allogeen','chr_renal_insuf','chron_dialysis','resp_insuf','cardio_vasc_insuf',
                    'cirrhosis','neoplasm','hem_malign','aids','imm_insuf',
                    'DiagnoseCode_last','age_at_admission', 'neutrophiles_count', 'nice_ap4_score','APACHE_IV_adm_diag_extracted','Survival_in_days', 'day_of_admission_max_count', 'mech_vent_max_count', 'rrt_max_count',
       'vasopr_inotr_max_count', 'transfusion_max_count', 'mort_90d', 'mort_1y']]

In [ ]:
# Concatenate the two DataFrames
samengevoegd_table_1_df = pd.concat([UMCU_table_1_df, AUMC_table_1_df], ignore_index=True)

In [ ]:
# Rename values in the 'Geslacht' column
samengevoegd_table_1_df['Geslacht'] = samengevoegd_table_1_df['Geslacht'].replace({
    'male': 'Male',
    'female': 'Female'
})

In [ ]:
# Define the list of values to rename
values_to_rename = [
    'Other specified and unspecified types of non-Hodgkin lymphoma',
    'Other specified types of T/NK-cell lymphoma',
    'Follicular lymphoma',
    'Hodgkin lymphoma',
    'Mature T/NK-cell lymphomas',
    'Malignant immunoproliferative diseases and certain other B-cell lymphomas',
    'Other and unspecified malignant neoplasms of lymphoid, hematopoietic and related tissue',
    'Other leukemias of specified cell type'
]

# Rename these values to 'Other' in the column 'DiagnoseCode_last'
samengevoegd_table_1_df['DiagnoseCode_last'] = samengevoegd_table_1_df['DiagnoseCode_last'].replace(values_to_rename, 'Other')


#### Make table 1

In [ ]:
# Define categorical and continuous variables
categorical_vars_1 = ['Geslacht','neutrophiles_smaller_than_0.5','autoloog','imm_insuf','allogeen','chr_renal_insuf','resp_insuf','cardio_vasc_insuf',
                    'cirrhosis',
                    'DiagnoseCode_last','APACHE_IV_adm_diag_extracted'
                    ]
continuous_vars_1 = ['age_at_admission', 'nice_ap4_score']
nonnormal_1=['neutrophiles_count', 'nice_ap4_score']


In [ ]:
rename_dict_1 = {'Geslacht':'Sex', 
               'neutrophiles_smaller_than_0.5':'Neutropenia',
               'autoloog':'Autologous SCT',
               'allogeen':'Allogenic SCT',
               'chr_renal_insuf':'Chronic renal insufficiency',
               'chron_dialysis':'Chronic dialysis',
               'resp_insuf':'Chronic respiratory insufficiency',
               'cardio_vasc_insuf':'Chronic cardiovascular insufficiency',
               'cirrhosis':'Cirrhosis',
               'neoplasm':'Metastatic neoplasm',
               'hem_malign':'Heamatological malignancy',
               'aids':'Aids',
               'imm_insuf':'Immunodeficiency',
               'APACHE_IV_adm_diag_extracted':'Admission diagnosis',
               'DiagnoseCode_last':'Heamatological malignancy',
               'age_at_admission':'Age',
               'neutrophiles_count':'Neutrophiles count',
               'nice_ap4_score':'APACHE IV score',
               'day_of_admission_max_count':'Length of stay',
               'mech_vent_max_count':'Length of IMV',
               'rrt_max_count':'Length of RRT',
               'vasopr_inotr_max_count':'Length of vasopression or inotropy',
               'transfusion_max_count':'Number of days with transfusion',
}

In [ ]:
limit_dict_1 = {'Geslacht':1,
              }

In [ ]:
samengevoegd_table_1_df['DiagnoseCode_last'].unique()

In [ ]:
order_dict_1={'DiagnoseCode_last':['Myeloid leukemia',
                                 'Non-follicular lymphoma',
                                 'Multiple myeloma and malignant plasma cell neoplasms',
                                 'Lymphoid leukemia',
                                 'Monocytic leukemia',
                                 'Leukemia of unspecified cell type',
                                 'Other',
]}

In [ ]:
groupby = ['malignant_hemat']  # Optional: Group by this variable

table_1_by_hemat = TableOne(samengevoegd_table_1_df, columns=['age_at_admission',
                                                              'Geslacht',
                                                              'cardio_vasc_insuf',
                                                              'resp_insuf',
                                                              'chr_renal_insuf',
                                                              'cirrhosis',
                                                              'imm_insuf',
                                                              'autoloog',
                                                              'allogeen',
                                                              'APACHE_IV_adm_diag_extracted',
                                                              'DiagnoseCode_last',
                                                              'neutrophiles_smaller_than_0.5',
                                                              'nice_ap4_score',], nonnormal=nonnormal_1, categorical=categorical_vars_1, 
                            groupby=groupby, pval=True, overall=False, missing=False, rename= rename_dict_1, limit=limit_dict_1, order= order_dict_1, htest_name=False)
# Print Table 1
print(table_1_by_hemat.tabulate(tablefmt = "fancy_grid"))

file_path = os.path.join(config.directory("tables"), "samengevoegd_table_1_by_hemat.xlsx")
table_1_by_hemat.to_excel(file_path)

# Make table 2

##### Make variables for organs support at any moment

In [ ]:
samengevoegd_table_1_df['mech_vent_at_any_point'] = (samengevoegd_table_1_df['mech_vent_max_count'] != 0).astype(int)
samengevoegd_table_1_df['rrt_at_any_point'] = (samengevoegd_table_1_df['rrt_max_count'] != 0).astype(int)
samengevoegd_table_1_df['vasopr_inotr_at_any_point'] = (samengevoegd_table_1_df['vasopr_inotr_max_count'] != 0).astype(int)
samengevoegd_table_1_df['transfusion_at_any_point'] = (samengevoegd_table_1_df['transfusion_max_count'] != 0).astype(int)


##### Make max_count variables with a Nan for the patients that have 0 days of that organ support

In [ ]:
samengevoegd_table_1_df['mech_vent_max_count_only_non_0'] = samengevoegd_table_1_df['mech_vent_max_count'].replace(0, np.nan)
samengevoegd_table_1_df['rrt_count_only_non_0'] = samengevoegd_table_1_df['rrt_max_count'].replace(0, np.nan)
samengevoegd_table_1_df['vasopr_inotr_max_count_only_non_0'] = samengevoegd_table_1_df['vasopr_inotr_max_count'].replace(0, np.nan)
samengevoegd_table_1_df['transfusion_max_count_only_non_0'] = samengevoegd_table_1_df['transfusion_max_count'].replace(0, np.nan)

In [ ]:
# Define categorical and continuous variables
categorical_vars_2 = ['mort_90d', 'mort_1y', 'mech_vent_at_any_point','rrt_at_any_point','vasopr_inotr_at_any_point','transfusion_at_any_point'
                    ]
continuous_vars_2 = ['day_of_admission_max_count', 'mech_vent_max_count_only_non_0', 'rrt_count_only_non_0',
       'vasopr_inotr_max_count_only_non_0', 'transfusion_max_count_only_non_0']
nonnormal_2=['day_of_admission_max_count', 'mech_vent_max_count_only_non_0', 'rrt_count_only_non_0',
       'vasopr_inotr_max_count_only_non_0', 'transfusion_max_count_only_non_0']

In [ ]:
rename_dict_2 = {'mort_90d':'90-day mortality', 
               'mort_1y':'1-year mortality',
               'mech_vent_at_any_point':'Mechanical ventilation at any moment',
               'rrt_at_any_point':'Renal replacement therapy at any moment',
               'vasopr_inotr_at_any_point':'Vasopressors or inotropes at any moment',
               'transfusion_at_any_point':'Transfusion of erythrocytes or platelets at any moment',
               'day_of_admission_max_count':'Length of ICU stay',
               'mech_vent_max_count_only_non_0':'Duration of mechanical ventilation',
               'rrt_count_only_non_0':'Duration of renal replacement therapy',
               'vasopr_inotr_max_count_only_non_0':'Duration of vasopressors or inotropes',
               'transfusion_max_count_only_non_0':'Duration of transfusion of erythrocytes or platelets'
}

In [ ]:
groupby = ['malignant_hemat']  # Optional: Group by this variable

table_2_by_hemat = TableOne(samengevoegd_table_1_df, columns=['day_of_admission_max_count',
                                                              'mech_vent_at_any_point',
                                                              'mech_vent_max_count_only_non_0',
                                                              'vasopr_inotr_at_any_point',
                                                              'vasopr_inotr_max_count_only_non_0',
                                                              'rrt_at_any_point',
                                                              'rrt_count_only_non_0',
                                                              'transfusion_at_any_point',
                                                              'transfusion_max_count_only_non_0',
                                                              'mort_90d',
                                                              'mort_1y',
                                                              ], nonnormal=nonnormal_2, categorical=categorical_vars_2, 
                            groupby=groupby, pval=True, overall=False, missing=False, rename= rename_dict_2, htest_name=False)
# Print Table 1
print(table_2_by_hemat.tabulate(tablefmt = "fancy_grid"))

file_path = os.path.join(config.directory("tables"), "samengevoegd_table_2_by_hemat.xlsx")
table_2_by_hemat.to_excel(file_path)